**Подготовка**

Все необходимые библиотеки загружаются на Gogle Drive, чтобы при перезапуске среды не приходилось заново их устанавливать.

*   youtube_dl - нужна для работы pafy
*   pafy - для скачивания аудио из видео с youtube
*   pydub - для работы с самим аудиофайлом: перевод в нужное расширение, деление файла на несколько маленьких по тишине
*   SpeechRecognition - для перевода аудио в текст

Следующие ячейки с установкой библиотек нужно запустить только один раз, далее при перезапуске они уже будут подключены.

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [ ]:
!pip install --target=$nb_path youtube_dl

In [ ]:
!pip install --target=$nb_path pafy

In [ ]:
!pip install --target=$nb_path SpeechRecognition

In [ ]:
!pip install --target=$nb_path pydub

**Получение аудио**

Данный код скачивает аудиодорожку у видео с youtube по ссылке и переименовывает в удобную строку.

In [ ]:
import pafy
import os
os.chdir('/content')
url = "https://www.youtube.com/watch?v=_7fsATsADIE"
result = pafy.new(url)
best_quality_audio = result.getbestaudio()
print(best_quality_audio)
best_quality_audio.download()
name = result.title
name = name.replace('/', '_')
name = name.replace('//', '__')
os.rename(name+'.webm', 'au.webm')

audio:webm@160k



**Деление аудио**

Сначала меняем расширение у файла, потому что с .webw работа не предусмотрена. Далее с помощью функции split_on_silnce() делим большую дорожку на много маленьких.

Аргументы функции:

*   song - файл с аудио, полученный с помощью функции AudioSegment.from_file()
*   min_silence_len - время тишины в миллисекундах, по которому делится аудио
*   silence_thresh - минимальная громкость 

После выолнения этого куска кода имеем список chunks, в котором хранятся маленькие аудио.


In [ ]:
#перевод в формат wav и деление аудио на отрезки по тишине
import speech_recognition as sr
from os import path
import os
import pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence

os.chdir('/content')
webw_audio = AudioSegment.from_file("au.webm", format="webm")
webw_audio.export("audio1.wav", format="wav")
#AUDIO_FILE = "audio1.wav"
path = "audio1.wav"
song = AudioSegment.from_wav(path)

chunks = split_on_silence(song, min_silence_len = 2000, silence_thresh = -19)
try:
    os.mkdir('audio_chunks6')
except(FileExistsError):
    pass

os.chdir('audio_chunks6')

CouldntDecodeError: ignored

**Скачивание аудио по предложениям и распознавание текста каждого**

В цикле сначала определяем кусочек аудио, добавляя к нему в начало и конец немного тишины, чтобы аудиофайлы не казались разрезанными. Далее с помощью функции recognize_google() распознаем текст. Распознавание от Google - самое качественное и удобное из тех, что предоставляет библиотека SpeechRecognition, поэтому выбрано именно оно. Запись распознанных предложений ведется в файлы.

In [ ]:
fh = open("recognized.txt", "w")
#os.chdir('audio_chunks6')
for i, chunk in enumerate(chunks):
    silence_chunk = AudioSegment.silent(duration=500)
    audio_chunk = silence_chunk + chunk + silence_chunk
    normalized_chunk = match_target_amplitude(audio_chunk, -20.0)
    #print("Exporting chunk{0}.wav.".format(i))
    normalized_chunk.export(".//chunk{0}.wav".format(i), bitrate = "192k", format = "wav")
    filename = 'chunk'+str(i)+'.wav'
    #print("Processing chunk "+str(i))
    file = filename
    r = sr.Recognizer()
    # распознать кусок
    with sr.AudioFile(file) as source:
        audio = r.record(source)
    try:
        # попробуйте преобразовать его в текст
        rec = r.recognize_google(audio, language="ru")
        # записать вывод в файл.
        fh.write(rec+'\n')
        #print(rec)
    except sr.UnknownValueError:
        pass
        #print("Could not understand audio")
    except sr.RequestError as e:
        pass
        #print("Could not request results. check your internet connection")
fh.close()

Exporting chunk0.wav.
Processing chunk 0
Добрый вечер фильм программа футбольный клуб в студию микрофона Максим курников а по видеосвязи Василию
Exporting chunk1.wav.
Processing chunk 1
естественно про группу и тут мне очень понравилась определение
Exporting chunk2.wav.
Processing chunk 2
тебя потому что насадки начала фамилии одинаковые назад
Exporting chunk3.wav.
Processing chunk 3
Ну так вот Дмитрий курицы знаешь пишет что групп
Exporting chunk4.wav.
Processing chunk 4
не знаю
Exporting chunk5.wav.
Processing chunk 5
Could not understand audio
Exporting chunk6.wav.
Processing chunk 6
по сравнению чтобы не выкладывай я вкладываю что придёт
Exporting chunk7.wav.
Processing chunk 7
по классу по уровню
Exporting chunk8.wav.
Processing chunk 8
Теперь у наших шансов больше потому что в кафе запретили курить
Exporting chunk9.wav.
Processing chunk 9
старых
Exporting chunk10.wav.
Processing chunk 10
тут
Exporting chunk11.wav.
Processing chunk 11
Could not understand audio
Exporting chunk12.w

In [ ]:
#pip install dostoevsky

In [ ]:
# from dostoevsky.tokenization import RegexTokenizer
# from dostoevsky.models import FastTextSocialNetworkModel
# tokenizer = RegexTokenizer()
# model = FastTextSocialNetworkModel(tokenizer=tokenizer)

# message = [
#     'хороших снов',
# ]

# results = model.predict(message, k=1)
# for mes, sentiment in zip(message, results):
#     for key in sentiment:
#         print(key)

ValueError: ignored